In [82]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [83]:
year_month = '202402'

In [84]:
year_month_dt = datetime.strptime(year_month, '%Y%m')

previous_year = year_month_dt.year - 1
previous_month = year_month_dt.month
previous_year_month = f"{previous_year:04d}{previous_month:02d}"

next_month_first_day = year_month_dt.replace(year=year_month_dt.year - 1, month=year_month_dt.month + 1)
etd = (next_month_first_day - timedelta(days=1)).strftime('%Y-%m-%d')

print(f"Previous year month: {previous_year_month}")
print(f"ETD (End of the month of the previous year): {etd}")


Previous year month: 202302
ETD (End of the month of the previous year): 2023-02-28


In [85]:
df_complaint_21_22 =pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\US Complaint Data 201901-202401.xlsx',sheet_name=1)
df_complaint_21_22 = df_complaint_21_22.loc[df_complaint_21_22['Year'].isin([2021,2022])]

C:\Users\ypeng\AppData\Local\Temp\ipykernel_3588\2844925773.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df_complaint_21_22 =pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\US Complaint Data 201901-202401.xlsx',sheet_name=1)


In [86]:
df_complaints_2024 = pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\2024\{} Complaint Data.xlsx'.format(year_month),sheet_name='2024 Complaint Database')
df_complaints_2024.drop(columns='New manufacturing complaints',inplace=True)

In [87]:
df_complaints_2023 = pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\2023\{} Complaint Data.xlsx'.format(previous_year_month),sheet_name='2023 Complaint Database')

In [88]:
df_complaints = pd.concat([df_complaints_2024,df_complaints_2023,df_complaint_21_22],ignore_index=True)

In [89]:
vendor_mapping = pd.read_excel(r'C:\Medline\2. CPM\data\vendor_mapping\Vendor _mapping 2024_v1.xlsx')
vendor_mapping['Vendor Number'] = vendor_mapping['Vendor Number'].map(str)

In [90]:
df_complaints['Notification Created Date'] = pd.to_datetime(df_complaints['Notification Created Date'],format='%Y-%m-%d')

In [91]:
(
    df_complaints
    .query("`If Manufacturing Complaint` == 'Y' and Year.isin([2021,2022,2023,2024])")
    .assign(Quarter =lambda d: d['Notification Created Date'].dt.quarter)
    .pivot_table(
        index = ['Division'],
        columns = ['Year','Quarter'],
        values = ['Notification Number'],
        aggfunc ='count',
        fill_value=0,
        margins=True
    )
)

Notification Number                                               \
Year                    2021                   2022                  2023   
Quarter                    1     2     3    4     1     2     3    4    1   
Division                                                                    
10                       158    71    69   39   109    84    69   60   37   
14                         7     6     5    0     0     6     2    2    0   
15                        52    35    28   15    33    34    27    2    6   
18                        19    23    17    3     5     9     9    3    1   
20                         1     7     5    2     1     3     1    0    2   
21                        15    17    16   10    17    19    14   11    5   
22                        80    59    34   35    29    29    34   23    5   
29                         2     0     3    1     3     2     2    0    0   
30                      1050   742   802  814   933   981  1239  729  385   
32                         0     0     0    0     0     0     0    0    0   
33                         0     2     1    3     6     0     0    0    0   
34                         0     0     0    0     0     0     0    0    0   
35                        98     1     0    0    32    33    12    2    1   
40                         8    10     9   12    12     5    19    9   11   
41                         4     4     4    1    12     5    12    2    0   
49                         0     0     0    0     0     0     0    0    0   
50                        35    10    13    4     1     2     1    7    2   
51                         2     0     3    6    14     7    16   11   14   
55                         0     1     2    2     1     4     0    1    0   
60                         0     1     0    0     0     0     1    0    0   
65                         0     0     0    0     0     0     0    0    0   
70                         0     0     0    0     0     1     0    0    0   
71                         0     1     0    0     0     0     2    0    0   
72                         0     0     0    0     5    11     4    2    0   
75                         3     0     0    0     0     0     0    1    2   
81                        39    34    45    7     0     0    10   25   10   
82                         3     1     0    1     2     1     0    0    2   
All                     1576  1025  1056  955  1215  1236  1474  890  483   

                      
Year     2024    All  
Quarter     1         
Division              
10         83    779  
14          2     30  
15         39    271  
18          7     96  
20          0     22  
21          9    133  
22         42    370  
29          0     13  
30        146   7821  
32         17     17  
33          0     12  
34         17     17  
35         36    215  
40         14    109  
41          1     45  
49          1      1  
50          1     76  
51          4     77  
55          0     11  
60          0      2  
65          3      3  
70          0      1  
71          0      3  
72          0     22  
75          6     12  
81          5    175  
82          0     10  
All       433  10343

In [92]:
df_complaints_gy = (
    df_complaints
    .query("`If Manufacturing Complaint` == 'Y' and Year.isin([2021,2022,2023,2024])")
    .assign(Quarter =lambda d: d['Notification Created Date'].dt.quarter,
            Division = lambda d: d['Division'].map(str),
            )
    .groupby(['Division','Year','Quarter','Month'],as_index=False).size()
)
df_complaints_gy.rename(columns={'size':'Complaints'},inplace=True)

In [93]:
df_BUOM_2021  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\US_ShippingData_2021.xlsx', usecols='D,E,AB,AP,BI')
df_BUOM_2022  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\US_ShippingData_2022.xlsx', usecols='D,E,AB,AP,BI')

In [94]:
df_BUOM_2023  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\US BUOM 202312.xlsx', usecols='C,D,H,P,Q')
df_BUOM_2023 = df_BUOM_2023.loc[df_BUOM_2023['ShipmentDt'] <= etd]


In [95]:
df_BUOM_2024  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\2024\US\US BUOM {}.xlsx'.format(year_month), usecols='D,E,P,X,Y')

In [96]:
df_BUOM_all = pd.concat([df_BUOM_2024,df_BUOM_2023,df_BUOM_2022,df_BUOM_2021],ignore_index=True)

In [97]:
df_BUOM_all.dropna(subset=['StrippedVendorNumber'],inplace=True)

In [98]:
df_BUOM_all['Year'] = df_BUOM_all['ShipmentDt'].dt.year
df_BUOM_all['Quarter'] = df_BUOM_all['ShipmentDt'].dt.quarter
df_BUOM_all['Month'] = df_BUOM_all['ShipmentDt'].dt.month

In [99]:
df_BUOM_all.rename(columns={'StrippedVendorNumber':'VendorNumber','ProductDivisionCode':'Division','BUOMShipmentQty':'BUOM'},inplace=True)
df_BUOM_all['Division'] = df_BUOM_all['Division'].map(int).map(str)
df_BUOM_all['VendorNumber'] = df_BUOM_all['VendorNumber'].map(int).map(str)

In [100]:
df_BUOM_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 645295 entries, 0 to 645297
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   VendorNumber  645295 non-null  object        
 1   VendorName    645295 non-null  object        
 2   Division      645295 non-null  object        
 3   BUOM          645295 non-null  int64         
 4   ShipmentDt    645295 non-null  datetime64[ns]
 5   Year          645295 non-null  int64         
 6   Quarter       645295 non-null  int64         
 7   Month         645295 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 44.3+ MB


In [101]:
df_BUOM_all

,VendorNumber,VendorName,Division,BUOM,ShipmentDt,Year,Quarter,Month
0,151886,HEBEI RUIYING TEXTILE CO LTD,75,864,2024-01-10,2024,1,1
1,151886,HEBEI RUIYING TEXTILE CO LTD,75,5832,2024-01-10,2024,1,1
2,151886,HEBEI RUIYING TEXTILE CO LTD,75,684,2024-01-10,2024,1,1
3,151886,HEBEI RUIYING TEXTILE CO LTD,75,7560,2024-01-10,2024,1,1
4,151886,HEBEI RUIYING TEXTILE CO LTD,75,432,2024-01-10,2024,1,1
...,...,...,...,...,...,...,...,...
645293,134137,APEX TRANSGULF MANUFACTURING,50,1200000,2021-12-29,2021,4,12
645294,148393,COMBO ZHONGSHAN MEDICAL EQUIPMENT C,30,1904,2021-12-20,2021,4,12
645295,105110,SINO PROTECTION HOLDINGS CO LTD,22,12740,2021-11-23,2021,4,11
645296,105110,SINO PROTECTION HOLDINGS CO LTD,22,1150,2021-11-23,2021,4,11


In [102]:
vendor_mapping_inspection = vendor_mapping.loc[(~vendor_mapping['Regional Manager'].isin(['Exemption','US vendor']))&(vendor_mapping['Regional Manager'].notnull()),'Vendor Number'].to_list()
df_BUOM_all = df_BUOM_all.loc[df_BUOM_all['VendorNumber'].isin(vendor_mapping_inspection)]

In [103]:
df_BUOM_gy = df_BUOM_all.groupby(['Division','Year','Quarter','Month'],as_index=False)['BUOM'].sum()

In [104]:
df_cpm = pd.merge(df_complaints_gy,df_BUOM_gy,on=['Division','Year','Quarter','Month'],how='outer').fillna(0)

In [105]:
df_cpm.to_excel('./out.xlsx')

In [106]:
pivot_table  = (
    df_cpm
    .pivot_table(
        index = ['Division'],
        columns = ['Year','Quarter'],
        values = ['Complaints', 'BUOM'], # 指定想要运算的列
        aggfunc = {'Complaints': 'sum', 'BUOM': 'sum'},
        fill_value = 0
    )
)


In [107]:
pivot_table.to_excel('pivot_table.xlsx')